# Is, slope, real

This exercise follows on from the previous "I, slope" exercise

In [ ]:
# Run this cell, but please don't change it.

# These lines import the Numpy and Pandas modules.
import numpy as np
# Make a random number generator.
rng = np.random.default_rng()

import pandas as pd
# Safe setting for Pandas.  Needs Pandas version >= 1.5.
pd.set_option('mode.copy_on_write', True)

from scipy.optimize import minimize

# These lines do some fancy plotting magic.
import matplotlib
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

# These lines load the tests.
from client.api.notebook import Notebook
ok = Notebook('is_slope_real.ok')

## Hemoglobin and serum creatinine, again

Again, we load the [data on chronic kidney disease](https://lisds.github.io/textbook/data/chronic_kidney_disease).

In [ ]:
ckd = pd.read_csv('ckd_clean.csv')
ckd.head()

Again, our interest here is in these two variables / columns:

* "Hemoglobin" : the concentration of the protein in red blood cells.  This
  tends to go down in chronic kidney disease.
* "Serum creatinine" : this is a measure of how well the kidney is clearing
  waste products from the blood.  If your kidneys are working well, your
  creatinine should be low.

In [ ]:
ckd.plot.scatter('Hemoglobin', 'Serum Creatinine')

In the [I, slope exercise](https://lisds.github.io/textbook/exercises/isloping.zip), you
found the best-fit slope to these data, in least-squares sense.

Looking at the plot, it seems as if there are two different things going on.

* To the left of the plot, there are scattered values for low hemoglobin and
  high creatinine, where it looks as if there is a weak straight line
  relationship between them.
* To the bottom right of the plot, there seems to be a tight cluster of values
  with high hemoglobin and creatinine, which it is difficult to see signs of
  a straight line relationship.

This data table has one row per patient.   The `Class` column divides the rows (patients) into two groups:

* Patients with chronic kidney disease have 1 in the `Class` column.
* Patients without chronic kidney disease have 0 in the `Class` column.

If you look at the "Class" values in the result of `ckd.head()` above, you'll
see that the first five rows in the data frame correspond to chronic kidney
disease patients.

In fact, the last five rows correspond to patients without chronic kidney
disease:

In [ ]:
# Show the last five rows of the data frame.
# Have a look at the values for the "Class" column.
ckd.tail()

You might correctly speculate that patients with chronic kidney disease tend
to have *both*:

* low hemoglobin (mostly because the kidney makes the hormone that stimulates
  the production of red blood cells, that contain hemoglobin)
* high creatinine (because the kidneys cannot clear creatinine from the blood).

Maybe the left part of the plot has the chronic kidney disease patients, and the right part has the patients without chronic kidney disease.

## Select patients with chronic kidney disease (CKD)

Make a new data frame called `ckd_patients` that contains only the patients labeled as having CKD.  Plot the Hemoglobin against Serum Creatinine for these patients.

In [ ]:
ckd_patients = ...
# Look at the last five values of the selected data frame.
ckd_patients.tail()

In [ ]:
_ = ok.grade('q_1_ckd_patients')

Convert these "Hemoglobin" and "Serum Creatinine" values to Numpy arrays, for
efficiency.

In [ ]:
hgb_ckd = ...
creat_ckd = ...

In [ ]:
_ = ok.grade('q_2_ckd_xy')

We are interested in the intercept and slope for this new line.  We start with
a scatter plot.  The `axis` call below makes sure that we see the 0,0 point on
the plot.

In [ ]:
# Plot the hemoglobin and creatinine for the CKD patients.
plt.scatter(hgb_ckd, creat_ckd)
plt.xlabel('Hemoglobin')
plt.ylabel('Creatinine')
# Make sure we have the 0,0 point on the plot.
plt.axis([0, np.max(hgb_ckd), 0, np.max(creat_ckd)])

Look at this plot, then make a guess for a good intercept and slope.

In [ ]:
guessed_intercept = ...
guessed_slope = ...

Now your job is to find the best (least-squares) line fitting `hgb_ckd` (on
the x axis) to `creat_ckd` (on the y axis).

Here we've helpfully copy-pasted the `rmse_any_line` function from [using
minimize](https://lisds.github.io/textbook/mean-slopes/using_minimize).

In [ ]:
def rmse_any_line(c_s, x_values, y_values):
    """ Root mean square error for line fitting y values to x values.

    Parameters
    ----------
    c_s : sequence
        Sequence, such as a list or array, with 2 elements.  The first is the
        intercept, the second is the slope.
    x_values : array
        Array of x values to predict from, using line defined in `c_s`.
    y_values : array
        Array of y values to predict, using line defined in `c_s`.

    Return
    ------
    rmse : float
        Square root of the mean squared error.
    """
    c, s = c_s
    predicted = c + x_values * s
    error = y_values - predicted
    return np.sqrt(np.mean(error ** 2))

Your next job is to use minimize, with this function, and return the results.
The results will include an attribute `x` that has the two-element array
containing the intercept and slope.

In [ ]:
# From your guesses above.
start_c_s = [guessed_intercept, guessed_slope]
# Now use the function and start guess to run "minimize"
results = minimize(rmse_any_line, start_c_s, ...)
# Show the returned results.
results

Finally fetch the intercept and slope that `minimize` found into their own
variables.

In [ ]:
best_c_ckd, best_s_ckd = ...
# Print the values.
print(best_c_ckd, best_s_ckd)

In [ ]:
_ = ok.grade('q_3_ls_ckd')

The next question is --- can we trust this slope?  Or could the slope the
negative have plausibly come about if we had drawn a random sample in a world
where there is no linear relationship between the Hemoglobin and the Serum
Creatinine, in the CKD patients.

Use permutation to calculate the *sampling distribution* of the slope.  See
[inference on slopes](https://lisds.github.io/textbook/mean-slopes/inference_on_slopes) for
inspiration.

As usual, let's start with a single *sample* (or trial) where we permute the y
values and find the best slope for this random permutation:

In [ ]:
shuffled_creat = rng.permutation(creat_ckd)
# Calculate the slope for this new permuted sequence of y values.
results = ...
# Show the results.
results.x

Now take 1000 samples for the sampling distribution. If you take the usual
10000 samples you will find the calculation takes a fairly long time.

In [ ]:
# To store samples for the sampling distribution
n_samples = 1000
fake_slopes = np.zeros(n_samples)
...
# Show a histogram of the sampling distribution
plt.hist(fake_slopes);

In [ ]:
_ = ok.grade('q_4_fake_slopes')

Calculate the proportion of the sampling distribution that is less than or equal to the observed best slope:

In [ ]:
p_slope = ...
# Show proportion.
p_slope

In [ ]:
_ = ok.grade('q_5_p_slope')

Looking back at the original plot of the CKD patients' values, we start to wonder whether the lowest and the highest values for Hemoglobin might be having an undue effect on the slope of the line.

Calculate the mean and standard deviation of the `hgb_ckd` values.

Make new arrays from which you have dropped the elements in `hgb_ckd` and
`creat_ckd` corresponding to Hemoglobin values lower than the mean minus 2.5
standard deviations, or higher than the mean plus 2.5 standard deviations.

Just to remind you, `np.std` gives you the standard deviation of an array.

In [ ]:
std_ckd = np.std(hgb_ckd)
std_ckd

*Hint*: consider `logical_and` or `np.abs`.

In [ ]:
hgb_clean = ...
creat_clean = ...
# Plot the new arrays.
plt.plot(hgb_clean, creat_clean, 'o')

In [ ]:
_ = ok.grade('q_6_cleaned')

Recalculate the best-fit slope for these arrays, from which you have dropped
the elements corresponding to extreme values.

In [ ]:
results = minimize(...)
best_c_clean, best_s_clean = ...
# Print the values.
print(best_c_clean, best_s_clean)

In [ ]:
_ = ok.grade('q_7_ls_clean')

Use permutation to recalculate the sampling distribution for this slope.
Again, calculate 1000 samples rather than the standard 10000, in order to save
time.


In [ ]:
# To store samples for the sampling distribution
n_samples = 1000
fake_slopes_clean = np.zeros(n_samples)
...
# Show a histogram of the sampling distribution
plt.hist(fake_slopes_clean);

In [ ]:
_ = ok.grade('q_8_fake_slopes_clean')

Calculate the proportion of the new sampling distribution that is less than or
equal to the observed best slope:

In [ ]:
p_slope_clean = ...
# Show proportion.
p_slope_clean

In [ ]:
_ = ok.grade('q_9_p_slope_clean')

Are you still convinced by the negative linear relationship between hemoglobin and serum creatinine?

## Done

You're finished with the assignment!  Be sure to...

- **run all the tests** (the next cell has a shortcut for that),
- **Save and Checkpoint** from the "File" menu.
- Finally, **restart** the kernel for this notebook, and **run all the cells**,
  to check that the notebook still works without errors.  Use the
  "Kernel" menu, and choose "Restart and run all".  If you find any
  problems, go back and fix them, save the notebook, and restart / run
  all again, before submitting.  When you do this, you make sure that
  we, your humble markers, will be able to mark your notebook.

In [ ]:
# For your convenience, you can run this cell to run all the tests at once!
import os
_ = [ok.grade(q[:-3]) for q in os.listdir("tests") if q.startswith('q')]